In [5]:
import requests 

/Users/dandyrahman/Documents/Projects/LLM-ZoomCamp/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Make list of documents

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Generate Document Id

In [7]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [8]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [9]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

### Checking Uniqueness of Document Id

In [10]:
from collections import defaultdict

In [11]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [12]:
len(hashes), len(documents)

(947, 948)

In [13]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [14]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

## Saving Documents to JSON

In [15]:
import json

In [16]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [17]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


## Generate Groundtruth Data using LLM

In [18]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [19]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"), #you can put the key here directy
)

In [20]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [21]:
from tqdm.auto import tqdm

/Users/dandyrahman/Documents/Projects/LLM-ZoomCamp/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
results = {}

In [23]:
count = 0 

for doc in tqdm(documents):
    if count == 10:
        break
    
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    count += 1

  1%|          | 10/948 [00:18<29:15,  1.87s/it]


In [53]:
results

{'c02e79ef': '[\n    "When will the course start?",\n    "What will happen on 15th Jan 2024 at 17h00?",\n    "How can I subscribe to the course public Google Calendar?",\n    "What do I need to do before the course starts?",\n    "How can I join the course Telegram channel and Slack channel?"\n]',
 '1f6520ca': '{\n    "question1": "What are the prerequisites for this course?",\n    "question2": "Is there a specific platform used for the course?",\n    "question3": "How can I access the course materials?",\n    "question4": "Are there any assignments or projects in the course?",\n    "question5": "Is there a discussion forum for students in the course?"\n}',
 '7842b56a': '[\n    "Is it possible to enroll in the course after the start date?",\n    "What will happen if I don\'t register for the course?",\n    "Am I allowed to submit the homeworks even if I join late?",\n    "Are there any consequences for turning in the final projects late?",\n    "Is it recommended to complete all assign

In [56]:
print(results['1f6520ca'])

{
    "question1": "What are the prerequisites for this course?",
    "question2": "Is there a specific platform used for the course?",
    "question3": "How can I access the course materials?",
    "question4": "Are there any assignments or projects in the course?",
    "question5": "Is there a discussion forum for students in the course?"
}


In [32]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [61]:
type(parsed_results['1f6520ca'])

dict

In [38]:
doc_index = {d['id']: d for d in documents}

In [44]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

## Saving Ground Truth Data to CSV

In [46]:
import pandas as pd

In [47]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [55]:
df['question'][5]

'question1'

In [62]:
df.to_csv('ground-truth-data.csv', index=False)

In [63]:
!head ground-truth-data.csv

question,course,document
When will the course start?,data-engineering-zoomcamp,c02e79ef
What will happen on 15th Jan 2024 at 17h00?,data-engineering-zoomcamp,c02e79ef
How can I subscribe to the course public Google Calendar?,data-engineering-zoomcamp,c02e79ef
What do I need to do before the course starts?,data-engineering-zoomcamp,c02e79ef
How can I join the course Telegram channel and Slack channel?,data-engineering-zoomcamp,c02e79ef
question1,data-engineering-zoomcamp,1f6520ca
question2,data-engineering-zoomcamp,1f6520ca
question3,data-engineering-zoomcamp,1f6520ca
question4,data-engineering-zoomcamp,1f6520ca
